# Exercise 3

## Group 
- **ID**: <your group ID>

- **Members**: 
    - <your name1>
    - <your name2>
    - <your name3>

## Hand-in
- Please hand in this notebook with your code implementation via Ilias 
- Please make sure that there is exactly **one** submission per group

## Task Description

In this exercise, you will implement Supervised Finetuning (SFT) for the pretrained GPT-2 model. You should use the `transformers` library to load the pretrained model and tokenizer. You will finetune the model on the `Alpaca` dataset, which is a collection of instruction-following examples. The dataset can be found [here](https://huggingface.co/datasets/tatsu-lab/alpaca).
Your implementation should contain the four parts specified below.

## Grading scheme
Total: 5 points
1. **Preparing the Dataloader** (1 point)
2. **Sensible Configurations** (1 point)
3. **Training loop** (2 points)
4. **Generation of Question Answer pairs** (1 point)